# Prompt Optimization Baseline
---

This notebook provides a baseline for the prompt optimization task.




### Import Dependencies

In [ ]:
import sys
import os
import yaml

sys.path.insert(0, "./")

import promptwizard
from promptwizard.glue.promptopt.instantiate import GluePromptOpt
from promptwizard.glue.promptopt.techniques.common_logic import (
    DatasetSpecificProcessing,
)
from promptwizard.glue.common.utils.file import save_jsonlist
from typing import Any
from tqdm import tqdm
from re import compile, findall
from datasets import load_dataset
from dotenv import load_dotenv

load_dotenv(override=True)


def update_yaml_file(file_path, config_dict):

    with open(file_path, "r") as file:
        data = yaml.safe_load(file)

    for field, value in config_dict.items():
        data[field] = value

    with open(file_path, "w") as file:
        yaml.dump(data, file, default_flow_style=False, allow_unicode=True)

    print("YAML file updated successfully!")

<br>

## 🧪 Case 1 : No training data. Don't want in-context examples in final prompt
---

The assumptions for this scenario are:

- No training data
- No in-context examples included in the final prompt

In [ ]:
path_to_config = "configs"
promptopt_config_path = os.path.join(path_to_config, "baseline_promptopt_config.yaml")
setup_config_path = os.path.join(path_to_config, "setup_config.yaml")

# Set the following based on the use case
config_dict = {
    "task_description": "You are a mathematics expert. You will be given a mathematics problem which you need to solve",
    "base_instruction": "Lets think step by step.",
    "mutation_rounds": 5,
}
update_yaml_file(promptopt_config_path, config_dict)

### Create an object for calling prompt optimization and inference functionalities

In [ ]:
gp = GluePromptOpt(
    promptopt_config_path, setup_config_path, dataset_jsonl=None, data_processor=None
)

### Call prompt optmization function
1. ```use_examples``` can be used when there are training samples and a mixture of real and synthetic in-context examples are required in the final prompt. When set to ```False``` all the in-context examples will be real
2. ```generate_synthetic_examples``` can be used when there are no training samples and we want to generate synthetic examples 
3. ```run_without_train_examples``` can be used when there are no training samples and in-context examples are not required in the final prompt 

In [ ]:
best_prompt, expert_profile = gp.get_best_prompt(
    use_examples=False,
    run_without_train_examples=True,
    generate_synthetic_examples=False,
)

<br>

## 🧪 Case 2: Have training data. Want in-context examples in final prompt
---

The assumptions for this scenario are:

- Have training data
- Have in-context examples and also want these in the final prompt

### AOAI client

In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv()

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not aoai_api_version:
    aoai_api_version = os.getenv("OPENAI_API_VERSION")
if not aoai_deployment_name:
    aoai_deployment_name = os.getenv("DEPLOYMENT_NAME")

try:
    client = AzureOpenAI(
        azure_endpoint=aoai_api_endpoint,
        api_key=aoai_api_key,
        api_version=aoai_api_version,
    )
    deployment_name = aoai_deployment_name
    print("=== Initialized AzuureOpenAI client ===")
    print(f"AZURE_OPENAI_ENDPOINT={aoai_api_endpoint}")
    print(f"AZURE_OPENAI_API_VERSION={aoai_api_version}")
    print(f"AZURE_OPENAI_DEPLOYMENT_NAME={aoai_deployment_name}")
except (ValueError, TypeError) as e:
    print(e)

### Transform dataset

In [ ]:
import json
from collections import OrderedDict


def summarize_answer(
    answer_text, deployment_name="gpt-4o-mini", temperature=0.1, max_tokens=500
):
    system_message = "You are a helpful assistant that summarizes answers in a concise and clear way."
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_message},
                {
                    "role": "user",
                    "content": f"Summarize the following answer:\n\n{answer_text}",
                },
            ],
            temperature=temperature,
            max_tokens=max_tokens,
        )

        summary = response.choices[0].message.content
        return summary
    except Exception as e:
        print(f"Error summarizing: {e}")
        return ""


def transform_jsonl(input_path, output_path, summarize=False):
    with open(input_path, "r", encoding="utf-8") as infile, open(
        output_path, "w", encoding="utf-8"
    ) as outfile:
        for line in infile:
            item = json.loads(line)
            new_item = OrderedDict()
            query = item.get("query", "")
            doc = item.get("document_content", "")
            ans = item.get("answer", "")

            if doc == "":
                doc_with_ans = ans
            else:
                doc_with_ans = (
                    f"<DOCUMENT_CONTENT>{doc}</DOCUMENT_CONTENT><ANSWER>{ans}</ANSWER>"
                )

            new_item["question"] = query
            new_item["answer"] = doc_with_ans
            if summarize:
                print(f"Summarizing answer: {ans[:30]}...")
                new_item["final_answer"] = summarize_answer(ans)
            else:
                new_item["final_answer"] = ans
            json.dump(new_item, outfile, ensure_ascii=False)
            outfile.write("\n")

In [ ]:
transform_jsonl(
    "dataset/mobile.jsonl", "dataset/mobile_transformed.jsonl", summarize=True
)

### Create a dataset specific class and define the required functions 

In [ ]:
class MyDataset(DatasetSpecificProcessing):

    def dataset_to_jsonl(self, dataset_jsonl: str, **kwargs: Any) -> None:
        def extract_answer_from_output(completion):

            return completion

        examples_set = []

        for _, sample in tqdm(enumerate(kwargs["dataset"]), desc="Evaluating samples"):
            example = {
                DatasetSpecificProcessing.QUESTION_LITERAL: sample[
                    "question"
                ],  # question
                DatasetSpecificProcessing.ANSWER_WITH_REASON_LITERAL: sample[
                    "answer"
                ],  # answer
                DatasetSpecificProcessing.FINAL_ANSWER_LITERAL: extract_answer_from_output(
                    sample["final_answer"]
                ),  # final_answer
            }
            examples_set.append(example)

        save_jsonlist(dataset_jsonl, examples_set, "w")

    def extract_final_answer(self, llm_output):

        return llm_output

In [ ]:
import os

path_to_config = "configs"
promptopt_config_path = os.path.join(path_to_config, "mobile_promptopt_config.yaml")
setup_config_path = os.path.join(path_to_config, "setup_config.yaml")

In [ ]:
answer_format = """Your response should be structured as follows:
- Product Specifications
- Product Design
- Product Summary
"""

config_dict = {"answer_format": answer_format, "unique_model_id": "gpt-4o"}
update_yaml_file(promptopt_config_path, config_dict)

### Create an object for calling prompt optimization and inference functionalities

In [ ]:
my_processor = MyDataset()

gp = GluePromptOpt(
    promptopt_config_path,
    setup_config_path,
    dataset_jsonl="dataset/mobile_transformed.jsonl",
    data_processor=my_processor,
)

### Call prompt optmization function
1. ```use_examples``` can be used when there are training samples and a mixture of real and synthetic in-context examples are required in the final prompt. When set to ```False``` all the in-context examples will be real
2. ```generate_synthetic_examples``` can be used when there are no training samples and we want to generate synthetic examples 
3. ```run_without_train_examples``` can be used when there are no training samples and in-context examples are not required in the final prompt 

**It will take a while to run.** Please make sure to set the ```use_examples``` and ```generate_synthetic_examples``` flags accordingly.

In [ ]:
best_prompt, expert_profile = gp.get_best_prompt(
    use_examples=True,
    run_without_train_examples=False,
    generate_synthetic_examples=False,
)

In [ ]:
print(f"Best prompt: {best_prompt} \nExpert profile: {expert_profile}")

### Save the best prompt and expert profile to a file

In [ ]:
import pickle

save_dir = "results"
if not os.path.exists(save_dir):
    os.system(f"mkdir {save_dir}")

with open(f"{save_dir}/mobile_best_prompt.pkl", "wb") as f:
    pickle.dump(best_prompt, f)
with open(f"{save_dir}/mobile_expert_profile.pkl", "wb") as f:
    pickle.dump(expert_profile, f)

In [ ]:
with open(f"{save_dir}/mobile_best_prompt.pkl", "rb") as f:
    loaded_best_prompt = pickle.load(f)
with open(f"{save_dir}/mobile_expert_profile.pkl", "rb") as f:
    loaded_expert_profile = pickle.load(f)